In [19]:
import sys, urllib, re, json, socket, string
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [15]:
train_ids = pd.read_csv("train_tweet_ids.tsv", sep="\t", header=None)
train_ids.head()

,0,1,2
0,333007591879884802,242520444,levaquin-518d891bac6ab35b4d481e06
1,344616533332467713,173701851,vyvanse-51b7c4c75378b9555a2f1ab8
2,344630199217958912,385562257,vyvanse-51b7d1815378b9555a2f1b0c
3,342117282585133056,1110475417,cymbalta-51aead545378f924d02efd7c
4,333102549362176001,356547564,seroquel-518de18bac6ab35b4d482156


In [133]:
train_ids.shape

(1340, 3)

In [161]:
train_anns = pd.read_csv("train_tweet_annotations.tsv", sep="\t", header=None)
train_anns.columns = ["id","start","stop","adr_present","adr_text","related_drug","target_drug"]
train_anns.head()

,id,start,stop,adr_present,adr_text,related_drug,target_drug
0,baclofen-518bf599ac6ab35b4d48099f,60,66,ADR,tired,baclofen,baclofen
1,baclofen-518bf599ac6ab35b4d48099f,67,73,ADR,sleepy,baclofen,baclofen
2,baclofen-518bf599ac6ab35b4d48099f,74,77,ADR,fog,baclofen,baclofen
3,baclofen-51905f6cac6ab35b4d483a24,69,75,ADR,gorked,baclofen,baclofen
4,baclofen-51ae70ba5378f924d02efb33,23,47,ADR,make me a bigger asshole,baclofen,baclofen


In [137]:
train_anns.shape

(991, 7)

In [139]:
len(train_anns[0].unique())

706

In [135]:
all_tweets = []
all_links = []
for i in range(train_ids.shape[0]):
    tweetid = train_ids[0][i]
    userid = train_ids[1][i]
    link = 'http://twitter.com/'+str(userid)+'/status/'+str(tweetid)
    all_links.append(link)
    request = requests.get(link).text
    soup = BeautifulSoup(request,"lxml")
    tweets = soup.find_all("p", "TweetTextSize--jumbo")
    tweet_text = list([x.get_text() for x in tweets])
    all_tweets.append(tweet_text)

In [148]:
train_ids_download = pd.concat([train_ids, pd.DataFrame(all_tweets), pd.DataFrame(all_links)], axis=1)
train_ids_download.columns = ["tweet_id", "user_id","id","tweet","url"]

In [149]:
train_ids_download.head()

,tweet_id,user_id,id,tweet,url
0,333007591879884802,242520444,levaquin-518d891bac6ab35b4d481e06,None,http://twitter.com/242520444/status/3330075918...
1,344616533332467713,173701851,vyvanse-51b7c4c75378b9555a2f1ab8,"Vyvanse, commonly known as OCD in a pill.",http://twitter.com/173701851/status/3446165333...
2,344630199217958912,385562257,vyvanse-51b7d1815378b9555a2f1b0c,@MTV When are you going to do True Life: I am ...,http://twitter.com/385562257/status/3446301992...
3,342117282585133056,1110475417,cymbalta-51aead545378f924d02efd7c,@upasbook Great read as always. I was on Cymba...,http://twitter.com/1110475417/status/342117282...
4,333102549362176001,356547564,seroquel-518de18bac6ab35b4d482156,None,http://twitter.com/356547564/status/3331025493...


In [155]:
train_ids_download.to_csv("train_ids_download.csv", index=False)

In [183]:
train_ids_valid = train_ids_download[train_ids_download["tweet"].isnull()==False].reset_index()
train_ids_valid.head()

,index,tweet_id,user_id,id,tweet,url
0,1,344616533332467713,173701851,vyvanse-51b7c4c75378b9555a2f1ab8,"Vyvanse, commonly known as OCD in a pill.",http://twitter.com/173701851/status/3446165333...
1,2,344630199217958912,385562257,vyvanse-51b7d1815378b9555a2f1b0c,@MTV When are you going to do True Life: I am ...,http://twitter.com/385562257/status/3446301992...
2,3,342117282585133056,1110475417,cymbalta-51aead545378f924d02efd7c,@upasbook Great read as always. I was on Cymba...,http://twitter.com/1110475417/status/342117282...
3,9,351446999637299200,1061562434,seroquel-51d09e2a53785f584a9ae686,I wonder if seroquel /then/ food still equals ...,http://twitter.com/1061562434/status/351446999...
4,12,348262745730338817,97567146,rivaroxaban-51c5089353785f584a9a91e5,Rivaroxaban diary day 22. Last tablet taken Tu...,http://twitter.com/97567146/status/34826274573...


In [185]:
#number of tweets that were able to found
train_ids_valid["id"].shape

(712,)

In [202]:
unique_anns = set(train_anns["id"])

In [195]:
adr_present = []
for i in range(train_ids_valid.shape[0]):
    if train_ids_valid["id"][i] in unique_anns:
        adr_present.append(1)
    else:
        adr_present.append(0)

In [198]:
train_ids_valid2 = pd.concat([train_ids_valid, pd.DataFrame(adr_present, columns=["adr_present"])], axis=1)
train_ids_valid2.head()

,index,tweet_id,user_id,id,tweet,url,adr_present
0,1,344616533332467713,173701851,vyvanse-51b7c4c75378b9555a2f1ab8,"Vyvanse, commonly known as OCD in a pill.",http://twitter.com/173701851/status/3446165333...,1
1,2,344630199217958912,385562257,vyvanse-51b7d1815378b9555a2f1b0c,@MTV When are you going to do True Life: I am ...,http://twitter.com/385562257/status/3446301992...,1
2,3,342117282585133056,1110475417,cymbalta-51aead545378f924d02efd7c,@upasbook Great read as always. I was on Cymba...,http://twitter.com/1110475417/status/342117282...,1
3,9,351446999637299200,1061562434,seroquel-51d09e2a53785f584a9ae686,I wonder if seroquel /then/ food still equals ...,http://twitter.com/1061562434/status/351446999...,1
4,12,348262745730338817,97567146,rivaroxaban-51c5089353785f584a9a91e5,Rivaroxaban diary day 22. Last tablet taken Tu...,http://twitter.com/97567146/status/34826274573...,1


In [204]:
train_ids_valid2.to_csv("train_ids_valid.csv", index=False)

In [206]:
train_ids_valid2["adr_present"].value_counts()

1    360
0    352
Name: adr_present, dtype: int64